In [1]:
from dataloader import load_quora
from heckman import weight_generating
import numpy as np
from dataloader import load_artificial_dataset
from math import fabs
from scipy.stats import norm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
import time


def save_weights(name, weight):
    np.save(open("./weights/weights_for_" + name + ".npy", "wb"), weight)


In [6]:
quora_data = load_quora()
weights = weight_generating(quora_data[0], quora_data[1], 1)
save_weights('quoraqp', weights)

---------- Loading QuoraQP ----------
Success!
---------- Generating weights ----------
Optimization terminated successfully.
         Current function value: nan
         Iterations 5
Optimization terminated successfully.
         Current function value: nan
         Iterations 5
Optimization terminated successfully.
         Current function value: nan
         Iterations 5
Optimization terminated successfully.
         Current function value: nan
         Iterations 5
Optimization terminated successfully.
         Current function value: nan
         Iterations 5
Optimization terminated successfully.
         Current function value: nan
         Iterations 5
Optimization terminated successfully.
         Current function value: nan
         Iterations 5
Optimization terminated successfully.
         Current function value: nan
         Iterations 5
Optimization terminated successfully.
         Current function value: nan
         Iterations 5
Optimization terminated successfully.
 

Computing inverse Mills ratios: 293761it [02:24, 2036.57it/s]


KeyboardInterrupt: 

In [2]:
loc = 4
scale = 4
artificial_data = load_artificial_dataset()
weights_list = [np.true_divide(norm.pdf(i, loc, scale), norm.cdf(i, loc, scale)) for i in artificial_data[0]]
weights_list = np.array(weights_list) / np.mean(np.array(weights_list))
save_weights('artificial_dataset', weights_list)

---------- Loading artificial dataset ----------
Success!


In [ ]:
import io
import re
from keras_preprocessing.text import Tokenizer
import pandas as pd
from keras.layers import Embedding
max_nb_words = 50000
emb_dim = 300
max_seq_len = 35
tr = pd.read_csv('./quora/train.tsv', delimiter='\t', header=None)
tr.columns = ['is_duplicate', 'question1', 'question2', 'pair_id']
tr = tr[['is_duplicate', 'question1', 'question2']]

val = pd.read_csv('./quora/dev.tsv', delimiter='\t', header=None)
val.columns = ['is_duplicate', 'question1', 'question2', 'pair_id']
val = val[['is_duplicate', 'question1', 'question2']]

tst = pd.read_csv('./quora/test.tsv', delimiter='\t', header=None)
tst.columns = ['is_duplicate', 'question1', 'question2', 'pair_id']
tst = tst[['is_duplicate', 'question1', 'question2']]
data = pd.concat([tr, val, tst], sort=False).fillna('')


def text_cleaning(text):
    text = re.sub('[^A-Za-z0-9]', ' ', text.lower())
    text = ' '.join(text.split())
    return text


data['question1'] = data['question1'].apply(text_cleaning)
data['question2'] = data['question2'].apply(text_cleaning)
tokenizer = Tokenizer(num_words=max_nb_words, oov_token='oov_token_placeholder')
tokenizer.fit_on_texts(list(data['question1'].values) + list(data['question2'].values))
sequences_1 = tokenizer.texts_to_sequences(data['question1'].values)
sequences_2 = tokenizer.texts_to_sequences(data['question2'].values)
word_index = tokenizer.word_index

word2vec = {}
fin = io.open('./quora/wordvec.txt', 'r', encoding='utf-8', newline='\n', errors='ignore')
for line in fin:
    tokens = line.rstrip().split(' ')
    word2vec[tokens[0]] = np.asarray(tokens[1:], dtype='float32')
print('Found %s word vectors of word2vec' % len(word2vec.keys()))
print('Preparing embedding matrix')
nb_words = min(max_nb_words, len(word_index))
emb = np.zeros((nb_words + 1, emb_dim))
miss_cnt = 0
for word, i in word_index.items():
    if i >= nb_words:
        break
    if word in word2vec.keys():
        emb[i] = word2vec[word]
    else:
        emb[i] = (np.random.rand(emb_dim) - 0.5) * 0.1
        miss_cnt += 1
print('Null word embeddings: %d' % miss_cnt)

embedding_layer = Embedding(emb.shape[0],
                                emb.shape[1],
                                weights=[emb],
                                trainable=False,
                                input_length=max_seq_len)
embedding_layer(data['question1'], data['question2'])


In [ ]:
X = np.array([artificial_data[0]]).T
y = artificial_data[1]
clf = RandomForestClassifier(
    n_estimators=10000, max_depth=9, random_state=2018, n_jobs=8, criterion='entropy')
time_start=time.time()
y_pred = cross_val_predict(clf, X, y, cv=100, method='predict_proba', verbose=3, n_jobs=3)
time_end = time.time()

In [21]:
propensity = np.array([y_pred[i, int(y[i])] for i in range(len(y))])
prob_1_l = np.array([(propensity[i] if y[i] == 1 else (1-propensity[i]))
          for i in range(len(y))])
prob_0_l = 1 - prob_1_l


def calculate_weight_fraction(prob_1):
    prob_0 = 1 - prob_1
    w1 = 1 / (prob_0 * prob_1_l / (prob_0 * prob_1_l + prob_1 * prob_0_l))
    w0 = 1 / (prob_1 * prob_0_l / (prob_0 * prob_1_l + prob_1 * prob_0_l))
    return sum(w1[i] for i in range(len(y)) if y[i] == 1) / sum(w0[i] for i in range(len(y)) if y[i] == 0)


prior_fraction = np.sum(y) / (len(y) - np.sum(y))
l, r = 0, 1
thr = 0.00000000001
step = 100
# while l + thr < r:
for _ in range(step):
    m1 = l + (r- l) / 2
    if calculate_weight_fraction(m1) < prior_fraction:
        l = m1
    else:
        r = m1

m0 = 1 - m1
w1 = 1 / (m0 * prob_1_l / (m0 * prob_1_l + m1 * prob_0_l))
w0 = 1 / (m1 * prob_0_l / (m0 * prob_1_l + m1 * prob_0_l))
weights = np.array([(w1[i] if y[i] == 1 else w0[i]) for i in range(len(y))])
assert fabs(prior_fraction - sum([weights[i] for i in range(len(y)) if y[i] == 1]) / sum([weights[i] for i in range(len(y)) if y[i] == 0])) < 0.0001
weights = np.true_divide(weights, np.mean(weights))
np.save(open("./weights/weights_artificial_random_forest.npy", "wb"), weights)
